# Загрзука модели

In [1]:
pip install  transformers datasets peft accelerate bitsandbytes evaluate sentencepiece


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 99.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 81.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 41.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig, get_peft_model, TaskType

model_name = "EleutherAI/gpt-neo-1.3B"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_8bit=True,
    device_map="auto"
)

lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    bias="none"
)
model = get_peft_model(model, lora_config)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors:   0%|          | 0.00/5.31G [00:00<?, ?B/s]



# Датасет и токенизация


In [3]:
import pandas as pd
from transformers import AutoTokenizer, DataCollatorForLanguageModeling
from datasets import Dataset

In [5]:
df = pd.read_csv('lenta-ru-news.csv.bz2')
df.head()

/tmp/ipython-input-5-3694259950.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('lenta-ru-news.csv.bz2')


,url,title,text,topic,tags,date
0,https://lenta.ru/news/1914/09/16/hungarnn/,1914. Русские войска вступили в пределы Венгрии,Бои у Сопоцкина и Друскеник закончились отступ...,Библиотека,Первая мировая,1914/09/16
1,https://lenta.ru/news/1914/09/16/lermontov/,1914. Празднование столетия М.Ю. Лермонтова от...,"Министерство народного просвещения, в виду про...",Библиотека,Первая мировая,1914/09/16
2,https://lenta.ru/news/1914/09/17/nesteroff/,1914. Das ist Nesteroff!,"Штабс-капитан П. Н. Нестеров на днях, увидев в...",Библиотека,Первая мировая,1914/09/17
3,https://lenta.ru/news/1914/09/17/bulldogn/,1914. Бульдог-гонец под Льежем,Фотограф-корреспондент Daily Mirror рассказыва...,Библиотека,Первая мировая,1914/09/17
4,https://lenta.ru/news/1914/09/18/zver/,1914. Под Люблином пойман швабский зверь,"Лица, приехавшие в Варшаву из Люблина, передаю...",Библиотека,Первая мировая,1914/09/18


In [6]:
texts = [f"{row['title']}\n\n{row['text']}" for _, row in df.iterrows() if pd.notna(row['text'])]
texts = texts[:10000]

In [7]:
print(texts[1])

1914. Празднование столетия М.Ю. Лермонтова отложено 

Министерство народного просвещения, в виду происходящих чрезвычайных событий, признало соответственным в день годовщины со дня рождения М.Ю. Лермонтова (2-го октября 1914 года) ограничиться совершением в учебных заведениях панихиды по поэту, отложив празднование юбилея до более благоприятного времени.  
«Русский инвалид», 16 сентября 1914 года.


In [8]:
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-1.3B")
tokenizer.pad_token = tokenizer.eos_token
dataset = Dataset.from_dict({"text": texts})

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
    pad_to_multiple_of=8
)
def tokenize(example):
    tokens = tokenizer(
        example["text"],
        truncation=True,
        padding="max_length",
        max_length=512
    )
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens


tokenized_dataset = dataset.map(tokenize, batched=True)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [9]:
dataset_split = tokenized_dataset.train_test_split(test_size=0.2, seed=42)
train_dataset = dataset_split["train"]
eval_dataset = dataset_split["test"]

In [10]:
train_dataset

Dataset({
    features: ['text', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 8000
})

In [11]:
eval_dataset[0]

{'text': 'Христенко не\xa0исключает реструктуризацию украинских долгов\n\nРоссия не исключает возможности реструктуризации долгов Украины за газ при условии, что она не будет накапливать новых. Об этом заявил в пятницу вице-премьер Виктор Христенко после встречи с министром иностранных дел Украины Борисом Тарасюком. По его словам, Россия готова перейти к чисто финансовым расчетам с Украиной за газ и его транспортировку. Однако "проблема в том, чтобы в этих операциях отсутствовал риск", заметил вице-премьер. Христенко считает, что риски можно снизить, если российский "Газпром" и его партнер "Нефтегаз Украины" будут участвовать в капитале друг друга. Для нас это очень важно, поскольку это будет гарантией при любой схеме расчетов, отметил он. Христенко также сообщил, что расхождений по сумме долга Украины за потребленный газ нет, а есть расхождения по его урегулированию. Как сообщалось ранее, Россия настаивала на сумме долга Украины за газ в 1,9 миллиарда долларов. Однако украинская сторо

# Fine-Tuning

In [12]:
!pip install transformers datasets accelerate peft bitsandbytes


In [11]:
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling
from transformers import AutoModelForCausalLM
from peft import LoraConfig, get_peft_model, TaskType

In [12]:
model_name = "EleutherAI/gpt-neo-1.3B"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_8bit=True,
    device_map="auto"
)


lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    task_type=TaskType.CAUSAL_LM
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


trainable params: 1,572,864 || all params: 1,317,148,672 || trainable%: 0.1194


In [13]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

training_args = TrainingArguments(
    output_dir="./gptneo-lora-checkpoints",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=1,
    logging_steps=50,
    save_strategy="epoch",
    eval_strategy="epoch",
    fp16=True,
    save_total_limit=1
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)


/tmp/ipython-input-13-930101868.py:15: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [14]:
trainer.train()


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: zheksembaev-temirlan (zheksembaev-temirlan-aitu) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:185: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:185: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Epoch,Training Loss,Validation Loss


/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:185: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:185: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:185: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:185: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during q

KeyboardInterrupt: 

In [ ]:
model.save_pretrained("neo-lenta-lora")
tokenizer.save_pretrained("neo-lenta-lora")


# Генерация текста

In [15]:
def generate_text(prompt, max_new_tokens=100):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=0.8,
        eos_token_id=tokenizer.eos_token_id
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


In [17]:
prompt = "Президент России выступил с заявлением о "
output = generate_text(prompt)
print(output)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Президент России выступил с заявлением о сделке по контролю над нефтегазовыми регулярными средствами депортации в Сирии

Президент Р


# Оценка Качества

In [18]:
from datasets import load_metric
import torch
import math
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="bitsandbytes")

eval_texts = eval_dataset.select(range(100))

def compute_perplexity(model, tokenizer, dataset):
    model.eval()
    total_loss = 0.0
    total_tokens = 0

    for example in dataset:
        input_ids = torch.tensor(example['input_ids']).unsqueeze(0).to(model.device)
        with torch.no_grad():
            outputs = model(input_ids, labels=input_ids)
            loss = outputs.loss
            total_loss += loss.item() * input_ids.size(1)
            total_tokens += input_ids.size(1)

    avg_loss = total_loss / total_tokens
    ppl = math.exp(avg_loss)
    return ppl

perplexity = compute_perplexity(model, tokenizer, eval_texts)
print(f"Perplexity: {perplexity:.2f}")


Perplexity: 2.42


In [21]:
from evaluate import load

rouge = load("rouge")
bleu = load("bleu")

sampled = eval_dataset.select(range(10))
prompts = [ex['text'][:50] for ex in sampled]
references = [ex['text'] for ex in sampled]

predictions = [generate_text(p) for p in prompts]

rouge_result = rouge.compute(predictions=predictions, references=references)
bleu_result = bleu.compute(predictions=predictions, references=[[r] for r in references])

print(" ROUGE-L:", rouge_result["rougeL"])
print(" BLEU:", bleu_result["bleu"])


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 ROUGE-L: 0.0
 BLEU: 3.1612661648489576e-06


Мы здесь видим низике показтели для BLEU и ROUGE-L. Но если вспомнить что они отвечают или скорее показываеют, то насколько предложенний ответ схож с референсом.То это объяснеяет их низкий показатель. И тем самым показываает что данные метрики не совсем уместны для данной задачи. На  примерах мы можем видеть, что модель отрабатывает хорошо и дает осмысленный ответ по промпту
Примеры прикрепил ниже:

In [22]:
for i in range(3):
    print(f"\nPrompt:\n{prompts[i]}")
    print(f"Prediction:\n{predictions[i]}")
    print(f"Reference:\n{references[i][:200]}...")


Prompt:
Христенко не исключает реструктуризацию украинских
Prediction:
Христенко не исключает реструктуризацию украинских активистов

В сентябре в Кремле обратился председатель народного региона Генеральной прокуратур
Reference:
Христенко не исключает реструктуризацию украинских долгов

Россия не исключает возможности реструктуризации долгов Украины за газ при условии, что она не будет накапливать новых. Об этом заявил в пятн...

Prompt:
Илларионов предупредил о спаде в экономике

Темпы 
Prediction:
Илларионов предупредил о спаде в экономике

Темпы реализации конденсата в иллюстрацию Илларионова связаны с оставлением финансовых программ. А
Reference:
Илларионов предупредил о спаде в экономике

Темпы экономического роста в России могут замедлиться, заявил во вторник журналистам советник президента РФ по связям с индустриально развитыми странами Анд...

Prompt:
Лужков выяснял отношения с московскими кавказцами

Prediction:
Лужков выяснял отношения с московскими кавказцами

Фото: Петр Па

In [23]:
from transformers import AutoModelForCausalLM

# базовая модель
base_model = AutoModelForCausalLM.from_pretrained("EleutherAI/gpt-neo-1.3B").to("cuda")

def generate_base(prompt, max_new_tokens=100):
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = base_model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=0.8,
        eos_token_id=tokenizer.eos_token_id
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

for i in range(3):
    print(f"\n Prompt:\n{prompts[i]}\n")

    print(" До обучения:")
    print(generate_base(prompts[i]))

    print("\n После обучения:")
    print(generate_text(prompts[i]))
    print("-" * 80)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



 Prompt:
Христенко не исключает реструктуризацию украинских

 До обучения:


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Христенко не исключает реструктуризацию украинских органов, которые нам упомянули в интервью «Известиям». В августе она отключила перехват в и

 После обучения:


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

Илларионов предупредил о спаде в экономике

Темпы сохранения и получения денежных рисков в экономическом положении или продлении коэффициент

 После обучения:


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Илларионов предупредил о спаде в экономике

Темпы сохранения экономических финансовых услуг в России составляют один спад в полтора года, в �
--------------------------------------------------------------------------------

 Prompt:
Лужков выяснял отношения с московскими кавказцами


 До обучения:


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Лужков выяснял отношения с московскими кавказцами

Лужков начал заниматься званиями кавказцами в постсоветском союзе

Автор фото, AFP Подпись к

 После обучения:
Лужков выяснял отношения с московскими кавказцами

С известными причинами прошлось критиковать выход из Москвы. Автор фотографии выяснил, что
--------------------------------------------------------------------------------
